## Gentrification across tracts from 2010 to 2018

In [ ]:
#Importing needed libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import math
import csv
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
#Reading the rent prioritisation data made available by the Urban Institute 
rent_data = gpd.read_file('/datasets/rent_prioritisation/housing_index_state_adj.geojson')

In [ ]:
#Filtering only the 15 cities
states = ['New York','California','Illinois','Michigan','Missouri','Texas','Georgia','Tennessee','Massachusetts','New Jersey','Minnesota','Florida']
counties = ['Kings County','Richmond County', 'Bronx County', ' Queens County', 'New York County',
'Los Angeles County','San Francisco County','Cook County','Wayne County','St. Louis County','Harris County',
'Dallas County','Fulton County','Davidson County','Suffolk County','Essex County','Hennepin County','Hillsborough County']
rent_data = rent_data[rent_data['state_name'].isin(states)]
rent_data = rent_data[rent_data['county_name'].isin(counties)]
rent_data.shape

(9589, 49)

In [ ]:
#Assigning city name in a new column
def city_name(row):
    if row['county_name'] == 'Kings County':
      val = 'New York'
    elif row['county_name'] == 'Richmond County':
      val = 'New York'
    elif row['county_name'] == 'Bronx County':
      val = 'New York'
    elif row['county_name'] == ' Queens County':
      val = 'New York'
    elif row['county_name'] == 'New York County':
      val = 'New York'
    elif row['county_name'] == 'Los Angeles County':
      val = 'Los Angeles'
    elif row['county_name'] == 'San Francisco County':
      val = 'San Francisco'
    elif row['county_name'] == 'Cook County':
      val = 'Chicago'
    elif row['county_name'] == 'Wayne County':
      val = 'Detroit'
    elif row['county_name'] == 'St. Louis County':
      val = 'St. Louis'
    elif row['county_name'] == 'Harris County':
      val = 'Houston'
    elif row['county_name'] == 'Dallas County':
      val = 'Dallas' 
    elif row['county_name'] == 'Fulton County':
      val = 'Atlanta'
    elif row['county_name'] == 'Davidson County':
      val = 'Nashville' 
    elif row['county_name'] == 'Suffolk County':
      val = 'Boston'
    elif row['county_name'] == 'Essex County':
      val = 'Newark'
    elif row['county_name'] == 'Hennepin County':
      val = 'Minneapolis' 
    elif row['county_name'] == 'Hillsborough County':
      val = 'Tampa' 
    else:
      val = 0

    return val

rent_data['city_name'] = rent_data.apply(city_name, axis=1)

In [ ]:
#Reading ACS 2010 and 2018 data
acs_2000 = pd.read_csv('/work/COVIDRedlining/ACS/ACS-Data-CensusTract/census_2000')
acs_2000 = acs_2000.drop(columns='Unnamed: 0')
acs_2018 = pd.read_csv('/work/COVIDRedlining/ACS/ACS-Data-CensusTract/acs_2018 (1)', usecols=['geo_id','total_pop','median_income','median_rent','owner_occupied_housing_units_median_value','bachelors_degree','white_pop','black_pop','hispanic_pop','asian_pop','amerindian_pop','other_race_pop','two_or_more_races_pop'])
acs_2018 = acs_2018.rename(columns = {'total_pop':'total_pop_18','median_income':'median_income_18','median_rent':'median_rent_18','owner_occupied_housing_units_median_value':'median_home_value_18','bachelors_degree':'bachelors_degree_18','white_pop':'white_pop_18','black_pop':'black_pop_18','hispanic_pop':'hispanic_pop_18','asian_pop':'asian_pop_18','amerindian_pop':'amerindian_pop_18','other_race_pop':'other_race_pop_18','two_or_more_races_pop':'two_or_more_races_pop_18'})
acs_2000.head()

,GEOID,total_population00,median_income00,median_home_value00,educational_attainment_00,white_alone00,black_alone00,native00,asian00,native_hawaiian_pacific_islander00,hispanic_or_latino00
0,34003001000,6683.181012,105711.9486,4.146347e+05,2726.303365,6034.576265,26.959115,4.989669,435.983661,0.0,169.665195
1,34003002100,2183.000000,130740.0000,1.000001e+06,961.000000,1647.000000,37.000000,6.000000,435.000000,0.0,55.000000
2,34003002200,4851.000000,74423.0000,2.515000e+05,1023.000000,3889.000000,56.000000,3.000000,657.000000,0.0,230.000000
3,34003002300,5751.000000,92447.0000,3.408000e+05,1669.000000,4356.000000,70.000000,1.000000,1122.000000,0.0,172.000000
4,34003003100,5057.000000,65750.0000,1.797000e+05,1021.000000,2880.000000,260.000000,10.000000,1067.000000,0.0,816.000000


In [ ]:
#Cleaning column names and types in ACS Data
acs_2000 = acs_2000.rename(columns={'geo_id':'GEOID'})
acs_2018 = acs_2018.rename(columns={'geo_id':'GEOID'})
acs_2000['GEOID'] = acs_2000['GEOID'].astype(float)
acs_2018['GEOID'] = acs_2018['GEOID'].astype(float)
rent_data['GEOID'] = rent_data['GEOID'].astype(float)

In [ ]:
#Merging ACS data with rent data
acs_2000 = rent_data.merge(acs_2000, on='GEOID',how='left')
acs_2018 = rent_data.merge(acs_2018, on='GEOID',how='left')

In [ ]:
acs_2000.columns

Index(['GEOID', 'state_name', 'county_name', 'state_fips', 'county_fips',
       'perc_cost_burdened_under_35k', 'perc_overcrowding_renter_1.50_or_more',
       'perc_unemployed_laborforce', 'perc_renters', 'perc_poverty_12mnth',
       'perc_no_hinsure', 'perc_person_of_color', 'perc_public_assistance',
       'perc_foreign_born', 'perc_30hamfi', 'perc_low_income_jobs_lost',
       'housing_index', 'covid_index', 'equity_index', 'total_index',
       'housing_index_quantile', 'covid_index_quantile',
       'equity_index_quantile', 'total_index_quantile',
       'z_score_perc_cost_burdened_under_35k',
       'z_score_perc_overcrowding_renter_1.50_or_more',
       'z_score_perc_unemployed_laborforce', 'z_score_perc_renters',
       'z_score_perc_poverty_12mnth', 'z_score_perc_no_hinsure',
       'z_score_perc_person_of_color', 'z_score_perc_public_assistance',
       'z_score_perc_foreign_born', 'z_score_perc_30hamfi',
       'z_score_perc_low_income_jobs_lost',
       'z_score_perc_cos

In [ ]:
# Merging both ACS files together
acs_data = pd.merge(acs_2018, acs_2000[['GEOID','total_population00','median_income00','median_home_value00','educational_attainment_00','white_alone00','black_alone00','native00','asian00','native_hawaiian_pacific_islander00','hispanic_or_latino00']], on='GEOID',how='left')
acs_data.head()

,GEOID,state_name,county_name,state_fips,county_fips,perc_cost_burdened_under_35k,perc_overcrowding_renter_1.50_or_more,perc_unemployed_laborforce,perc_renters,perc_poverty_12mnth,...,total_population00,median_income00,median_home_value00,educational_attainment_00,white_alone00,black_alone00,native00,asian00,native_hawaiian_pacific_islander00,hispanic_or_latino00
0,6.037543e+09,California,Los Angeles County,06,06037,1.000000,0.000000,0.067121,0.142857,0.089995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6.037544e+09,California,Los Angeles County,06,06037,0.811828,0.115824,0.070624,0.441007,0.215167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.037550e+09,California,Los Angeles County,06,06037,1.000000,0.000000,0.046247,0.182397,0.097878,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.037550e+09,California,Los Angeles County,06,06037,0.831858,0.060367,0.035970,0.328732,0.039739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.037551e+09,California,Los Angeles County,06,06037,0.473684,0.025373,0.077564,0.303442,0.026775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Only considering gentrification in boston

sf_acs = acs_data[(acs_data['city_name']=='San Francisco')]
sf_acs.head()

,GEOID,state_name,county_name,state_fips,county_fips,perc_cost_burdened_under_35k,perc_overcrowding_renter_1.50_or_more,perc_unemployed_laborforce,perc_renters,perc_poverty_12mnth,...,total_population00,median_income00,median_home_value00,educational_attainment_00,white_alone00,black_alone00,native00,asian00,native_hawaiian_pacific_islander00,hispanic_or_latino00
35,6.075016e+09,California,San Francisco County,06,06075,0.649231,0.019849,0.018450,0.729655,0.146853,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,6.075010e+09,California,San Francisco County,06,06075,0.554167,0.000000,0.007219,0.642267,0.052570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,6.075011e+09,California,San Francisco County,06,06075,0.750000,0.037257,0.044929,0.751014,0.135185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,6.075033e+09,California,San Francisco County,06,06075,1.000000,0.000000,0.052178,0.168785,0.061872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,6.075040e+09,California,San Francisco County,06,06075,0.911894,0.065975,0.038734,0.581716,0.054683,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
sf_acs.state_name.value_counts()

California    196
Name: state_name, dtype: int64

In [ ]:
sf_acs['median_home_value00'] = sf_acs['median_home_value00'].replace(0, pd.np.nan)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.
/opt/venv/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
#Calculating change in educational attainment
sf_acs['educational_attainment_pct'] = (
    100 * sf_acs['bachelors_degree_18'] /
    sf_acs['total_pop_18']
)

sf_acs['educational_attainment_pct00'] = (
    100 * sf_acs['educational_attainment_00'] / 
    sf_acs['total_population00']
)

sf_acs['educational_attainment_change'] = (
    sf_acs['educational_attainment_pct'] - 
    sf_acs['educational_attainment_pct00']
)

/opt/venv/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
#Getting inflation rate for 2000
inflation_rate  = 1.46

In [ ]:
#Getting median income and house value for the botton 40th percentile amongst all census tracts
median_home_2000_bottom_40th = sf_acs['median_home_value00'].quantile(0.4)

median_income_2000_bottom_40th = sf_acs['median_income00'].quantile(0.4)

sf_acs['home_pct_change'] = (
    (
        sf_acs['median_home_value_18'] - # current median home value
        (sf_acs['median_home_value00'] * inflation_rate)  # previous median home value adjusted for inflation
    ) / 
    sf_acs['median_home_value00'] # current median home value 
)

median_home_change_top3rd = sf_acs[sf_acs['home_pct_change'] !=None ]['home_pct_change'].quantile(0.66)

print("Bottom 40th percentile household income threshold for metro level data: " , median_home_2000_bottom_40th)
print("Bottom 40th percentile median home value for metro level data: " , median_income_2000_bottom_40th)
print("Top third percentile for all-metro-level percentage increase in a tract’s inflation-adjusted median home value: " , median_home_change_top3rd)

Bottom 40th percentile household income threshold for metro level data:  nan
Bottom 40th percentile median home value for metro level data:  nan
Top third percentile for all-metro-level percentage increase in a tract’s inflation-adjusted median home value:  nan
/opt/venv/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
#Considering top quantiles of educational attainment change
educational_attainment_threshold = boston_acs['educational_attainment_change'].quantile(2.0/3)
print(boston_acs['educational_attainment_change'].min())
print(boston_acs['educational_attainment_change'].max())
print(educational_attainment_threshold)

nan
nan
nan


In [ ]:
# The tract had a population of at least 500 residents at the beginning and 
# end of a decade and was located within a central city
boston_acs['has_pop_500+_2000'] = boston_acs['total_population00'] >= 500
boston_acs['has_pop_500+_2018'] = boston_acs['total_pop_18'] >= 500

# The tract’s median household income was in the bottom 40th percentile when compared 
# to all tracts within its metro area at the beginning of the decade.
boston_acs['median_40th_home_2000'] = boston_acs['median_home_value00'] < median_home_2000_bottom_40th

# The tract’s median home value was in the bottom 40th percentile when compared to all tracts within its metro area at the beginning of the decade.
boston_acs['median_40th_income_2000'] = boston_acs['median_income00'] < median_income_2000_bottom_40th
boston_acs.head(5)

/opt/venv/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


,GEOID,state_name,county_name,state_fips,county_fips,perc_cost_burdened_under_35k,perc_overcrowding_renter_1.50_or_more,perc_unemployed_laborforce,perc_renters,perc_poverty_12mnth,...,native_hawaiian_pacific_islander00,hispanic_or_latino00,educational_attainment_pct,educational_attainment_pct00,educational_attainment_change,home_pct_change,has_pop_500+_2000,has_pop_500+_2018,median_40th_home_2000,median_40th_income_2000
4485,2.502506e+10,Massachusetts,Suffolk County,25,25025,0.406504,0.011423,0.030358,0.480299,0.033161,...,NaN,NaN,43.158929,NaN,NaN,NaN,False,True,False,False
4486,2.502506e+10,Massachusetts,Suffolk County,25,25025,0.150652,0.008287,0.228942,0.992687,0.566957,...,NaN,NaN,9.826087,NaN,NaN,NaN,False,True,False,False
4487,2.502507e+10,Massachusetts,Suffolk County,25,25025,0.375661,0.043317,0.074512,0.693959,0.214014,...,NaN,NaN,19.335689,NaN,NaN,NaN,False,True,False,False
4488,2.502507e+10,Massachusetts,Suffolk County,25,25025,0.250441,0.029224,0.078431,0.963908,0.326531,...,NaN,NaN,16.263553,NaN,NaN,NaN,False,True,False,False
4489,2.502507e+10,Massachusetts,Suffolk County,25,25025,0.350606,0.042940,0.066608,0.628833,0.273875,...,NaN,NaN,24.358974,NaN,NaN,NaN,False,True,False,False


In [ ]:
# An increase in a tract's educational attainment, as measured by the 
# percentage of residents age 25 and over holding bachelor’s degrees, 
# was in the top third percentile of all tracts within a metro area.
boston_acs['educational_is_upper3rd'] = boston_acs['educational_attainment_change'] > educational_attainment_threshold

# The median home value increased, after accounting for inflation
boston_acs['median_home_higher'] = boston_acs['median_home_value_18'] > (boston_acs['median_home_value00'] * inflation_rate)

# The percentage increase in a tract’s inflation-adjusted median home 
# value was in the top third percentile of all tracts within a metro area.
boston_acs['median_home_top_third'] = boston_acs['home_pct_change'] > median_home_change_top3rd

boston_acs.head()

/opt/venv/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


,GEOID,state_name,county_name,state_fips,county_fips,perc_cost_burdened_under_35k,perc_overcrowding_renter_1.50_or_more,perc_unemployed_laborforce,perc_renters,perc_poverty_12mnth,...,educational_attainment_pct00,educational_attainment_change,home_pct_change,has_pop_500+_2000,has_pop_500+_2018,median_40th_home_2000,median_40th_income_2000,educational_is_upper3rd,median_home_higher,median_home_top_third
4485,2.502506e+10,Massachusetts,Suffolk County,25,25025,0.406504,0.011423,0.030358,0.480299,0.033161,...,NaN,NaN,NaN,False,True,False,False,False,False,False
4486,2.502506e+10,Massachusetts,Suffolk County,25,25025,0.150652,0.008287,0.228942,0.992687,0.566957,...,NaN,NaN,NaN,False,True,False,False,False,False,False
4487,2.502507e+10,Massachusetts,Suffolk County,25,25025,0.375661,0.043317,0.074512,0.693959,0.214014,...,NaN,NaN,NaN,False,True,False,False,False,False,False
4488,2.502507e+10,Massachusetts,Suffolk County,25,25025,0.250441,0.029224,0.078431,0.963908,0.326531,...,NaN,NaN,NaN,False,True,False,False,False,False,False
4489,2.502507e+10,Massachusetts,Suffolk County,25,25025,0.350606,0.042940,0.066608,0.628833,0.273875,...,NaN,NaN,NaN,False,True,False,False,False,False,False


In [ ]:
boston_acs['gentrified'] = (
    (boston_acs['has_pop_500+_2000'] == True) & 
    (boston_acs['has_pop_500+_2018'] == True) & 
    (boston_acs['median_40th_home_2000'] == True) & 
    (boston_acs['median_40th_income_2000'] == True) & 
    (boston_acs['median_home_higher'] == True) & 
    (boston_acs['median_home_top_third'] == True)
)

/opt/venv/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
boston_acs['pct_white_alone_change'] = ((boston_acs['white_pop_18']/boston_acs['total_population00'])*100)-((boston_acs['white_alone00']/boston_acs['total_population00'])*100)
boston_acs['pct_white_alone_2018']= ((boston_acs['white_pop_18']/boston_acs['total_pop_18'])*100)

boston_acs['pct_black_alone_change'] = ((boston_acs['black_pop_18']/boston_acs['total_pop_18'])*100)-((boston_acs['black_alone00']/boston_acs['total_population00'])*100)
boston_acs['pct_black_alone_2018'] = ((boston_acs['black_pop_18']/boston_acs['total_pop_18'])*100)

boston_acs['pct_asian_alone_change'] = ((boston_acs['asian_pop_18']/boston_acs['total_pop_18'])*100)-((boston_acs['asian00']/boston_acs['total_population00'])*100)
boston_acs['pct_asian_alone_2018'] = ((boston_acs['asian_pop_18']/boston_acs['total_pop_18'])*100)

boston_acs['pct_hispanic_alone_change'] = ((boston_acs['hispanic_pop_18']/boston_acs['total_pop_18'])*100)-((boston_acs['hispanic_or_latino00']/boston_acs['total_population00'])*100)
boston_acs['pct_hispanic_alone_2018'] = ((boston_acs['hispanic_pop_18']/boston_acs['total_pop_18'])*100)

/opt/venv/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
boston_acs["low_population"] = (boston_acs["total_population00"] < 500) | (boston_acs["total_pop_18"] < 500)

/opt/venv/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [ ]:
boston_acs.head()

,GEOID,state_name,county_name,state_fips,county_fips,perc_cost_burdened_under_35k,perc_overcrowding_renter_1.50_or_more,perc_unemployed_laborforce,perc_renters,perc_poverty_12mnth,...,gentrified,pct_white_alone_change,pct_white_alone_2018,pct_black_alone_change,pct_black_alone_2018,pct_asian_alone_change,pct_asian_alone_2018,pct_hispanic_alone_change,pct_hispanic_alone_2018,low_population
4485,2.502506e+10,Massachusetts,Suffolk County,25,25025,0.406504,0.011423,0.030358,0.480299,0.033161,...,False,NaN,91.378040,NaN,0.663228,NaN,4.151314,NaN,2.579219,False
4486,2.502506e+10,Massachusetts,Suffolk County,25,25025,0.150652,0.008287,0.228942,0.992687,0.566957,...,False,NaN,17.739130,NaN,19.956522,NaN,17.260870,NaN,41.260870,False
4487,2.502507e+10,Massachusetts,Suffolk County,25,25025,0.375661,0.043317,0.074512,0.693959,0.214014,...,False,NaN,52.522968,NaN,2.318021,NaN,34.586572,NaN,8.381625,False
4488,2.502507e+10,Massachusetts,Suffolk County,25,25025,0.250441,0.029224,0.078431,0.963908,0.326531,...,False,NaN,29.816514,NaN,12.260217,NaN,47.372811,NaN,7.881568,False
4489,2.502507e+10,Massachusetts,Suffolk County,25,25025,0.350606,0.042940,0.066608,0.628833,0.273875,...,False,NaN,47.832168,NaN,11.655012,NaN,18.671329,NaN,14.778555,False


In [ ]:
boston_acs.gentrified.value_counts()

False    203
Name: gentrified, dtype: int64

In [ ]:
boston_acs[boston_acs['gentrified']==False][['home_pct_change','educational_attainment_change','pct_white_alone_change','pct_black_alone_change','pct_asian_alone_change','pct_hispanic_alone_change']]

,home_pct_change,educational_attainment_change,pct_white_alone_change,pct_black_alone_change,pct_asian_alone_change,pct_hispanic_alone_change
4485,NaN,NaN,NaN,NaN,NaN,NaN
4486,NaN,NaN,NaN,NaN,NaN,NaN
4487,NaN,NaN,NaN,NaN,NaN,NaN
4488,NaN,NaN,NaN,NaN,NaN,NaN
4489,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
4845,NaN,NaN,NaN,NaN,NaN,NaN
4846,NaN,NaN,NaN,NaN,NaN,NaN
4847,NaN,NaN,NaN,NaN,NaN,NaN
4848,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
boston_acs.columns

Index(['GEOID', 'state_name', 'county_name', 'state_fips', 'county_fips',
       'perc_cost_burdened_under_35k', 'perc_overcrowding_renter_1.50_or_more',
       'perc_unemployed_laborforce', 'perc_renters', 'perc_poverty_12mnth',
       'perc_no_hinsure', 'perc_person_of_color', 'perc_public_assistance',
       'perc_foreign_born', 'perc_30hamfi', 'perc_low_income_jobs_lost',
       'housing_index', 'covid_index', 'equity_index', 'total_index',
       'housing_index_quantile', 'covid_index_quantile',
       'equity_index_quantile', 'total_index_quantile',
       'z_score_perc_cost_burdened_under_35k',
       'z_score_perc_overcrowding_renter_1.50_or_more',
       'z_score_perc_unemployed_laborforce', 'z_score_perc_renters',
       'z_score_perc_poverty_12mnth', 'z_score_perc_no_hinsure',
       'z_score_perc_person_of_color', 'z_score_perc_public_assistance',
       'z_score_perc_foreign_born', 'z_score_perc_30hamfi',
       'z_score_perc_low_income_jobs_lost',
       'z_score_perc_cos